## 🔐 Step 1 - Mounting Google Drive

Run the code below to authorize Colab to access your Google Drive.

Once mounted, your files will be accessible at:  
`/content/drive/MyDrive/`

---

## 🔐 Etapa 1 - Montagem do Google Drive

Execute o código abaixo para autorizar o acesso do Colab ao seu Google Drive.

Após montar, seus arquivos estarão acessíveis pelo caminho:  
`/content/drive/MyDrive/`

> ⚠️ Mesmo com o idioma da conta em português, o caminho técnico utilizado pelo Colab é `"MyDrive"`.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 🔧 Step 2 - File Handling Functions

Here we define helper functions that will be used in the file organization process:

- Classify files by type;
- Classify files by size (small, medium, large, huge);
- Get the date of last modification;
- Dynamically create folders based on the chosen criteria;
- Move files to the appropriate folders.

These functions are used by the main organization functions, which are described in the next step.

---

## 🔧 Etapa 2 - Funções para tratar arquivos

Aqui definimos funções auxiliares que serão usadas no processo de organização:

- Classificar arquivos por tipo;
- Classificar arquivos por tamanho (pequenos, médios, grandes, enormes);
- Obter a data da última modificação;
- Criar pastas dinamicamente conforme os critérios escolhidos;
- Mover os arquivos para as pastas corretas.

Essas funções são utilizadas pelas funções principais de organização, descritas na próxima etapa.


In [ ]:
import os, shutil, datetime

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"📁 Folder created: {path}")

def move_file(file_path, dest_folder):
    try:
        shutil.move(file_path, dest_folder)
        print(f"✅ '{os.path.basename(file_path)}' moved to '{dest_folder}'")
    except Exception as e:
        print(f"❌ Error while moving '{file_path}': {e}")

## 🧠 Step 3 - Main Organization Functions (by separate criteria)

The main organization functions are separated by criteria, allowing greater control and clarity in the process:

- `organize_by_type()`: groups files into categories such as Images, Documents, Videos, Music, and Others, based on the file type identified by the name.

- `organize_by_size()`: groups files into folders by size range: Small (<1MB), Medium, Large, Huge (>100MB).

- `organize_by_date()`: creates subfolders in the format YYYY-MM based on the last modification date of each file.

- `organize_by_keyword()`: groups files based on user-defined keywords found in the file names (e.g., "exam", "list", "assignment").

> ⚠️ Although it is technically possible to run multiple organization functions in sequence, it is not recommended. Each function moves files into a new structure, which can disrupt the previous organization.

> If you want to apply more than one criterion, it’s best to do it manually and in a planned way, or use a custom function that combines multiple criteria hierarchically (e.g., type → date).

> 📂 You can and should choose a specific folder (and its subfolders) to organize. You don’t need to apply the function to the root of your Drive (`/MyDrive`), unless you want to reorganize everything. All files within the selected folder and its subfolders will be reorganized based on the chosen criterion.

---

## 🧠 Etapa 3 - Funções principais de organização (por critério separado)

As funções principais de organização estão separadas por critério, permitindo maior controle e clareza no processo:

- `organizar_por_tipo()`: agrupa arquivos em categorias como Imagens, Documentos, Vídeos, Músicas e Outros, com base no tipo de arquivo identificado pelo nome.

- `organizar_por_tamanho()`: agrupa arquivos em pastas por faixa de tamanho: Pequenos (<1MB), Médios, Grandes, Gigantes (>100MB).

- `organizar_por_data()`: cria subpastas no formato AAAA-MM com base na data da última modificação de cada arquivo.

- `organizar_por_palavra_chave()`: agrupa arquivos com base em palavras-chave definidas pelo usuário no nome dos arquivos (ex: "prova", "lista", "trabalho").

> ⚠️ Embora seja tecnicamente possível executar duas ou mais funções de organização em sequência, não é recomendado. Isso porque cada função move os arquivos para uma estrutura diferente, o que pode quebrar a organização feita anteriormente.

> Se quiser aplicar mais de um critério, o ideal é fazer isso de forma planejada e manual, ou utilizar uma função personalizada que combine os critérios hierarquicamente (ex: tipo → data).

> 📂 Você pode (e deve) escolher uma pasta específica para organizar (e suas subpastas). Não é necessário aplicar as funções na raiz do seu Drive (`/MyDrive`), a menos que deseje reorganizar absolutamente tudo. Todos os arquivos da pasta selecionada e suas subpastas serão reorganizados com base no critério escolhido.


In [ ]:
def identify_type(file_name):
    file_type = file_name.lower()
    if any(file_type.endswith(ext) for ext in ['jpg', 'jpeg', 'png', 'gif']): return 'Images'
    elif any(file_type.endswith(ext) for ext in ['pdf', 'docx', 'doc', 'txt', 'xlsx', 'xls', 'gdoc', 'gsheet', 'gslides']): return 'Documents'
    elif any(file_type.endswith(ext) for ext in ['mp4', 'mov', 'avi']): return 'Videos'
    elif any(file_type.endswith(ext) for ext in ['mp3', 'wav']): return 'Music'
    else:
        return 'Others'

def organize_by_type(path):
    for root, _, files in os.walk(path):
        if any(root.endswith(p) for p in ['Images', 'Documents', 'Videos', 'Music', 'Others']): continue
        for file in files:
            full_path = os.path.join(root, file)
            if os.path.isfile(full_path):
                file_type = identify_type(file)
                destination = os.path.join(path, file_type)
                create_folder(destination)
                move_file(full_path, destination)

In [ ]:
def classify_size(size_bytes):
    mb = size_bytes / (1024 * 1024)
    if mb < 1: return 'Small'
    elif mb < 10: return 'Medium'
    elif mb < 100: return 'Large'
    else:
        return 'Huge'

def organize_by_size(path):
    for root, _, files in os.walk(path):
        if any(root.endswith(p) for p in ['Small', 'Medium', 'Large', 'Huge']): continue
        for file in files:
            full_path = os.path.join(root, file)
            if os.path.isfile(full_path):
                size_category = classify_size(os.path.getsize(full_path))
                destination = os.path.join(path, size_category)
                create_folder(destination)
                move_file(full_path, destination)

In [ ]:
def get_date_folder(full_path):
    modified_at = datetime.datetime.fromtimestamp(os.path.getmtime(full_path))
    return modified_at.strftime('%Y-%m')

def organize_by_date(path):
    for root, _, files in os.walk(path):
        if any(len(folder) == 7 and folder[4] == '-' for folder in root.split(os.sep)): continue
        for file in files:
            full_path = os.path.join(root, file)
            if os.path.isfile(full_path):
                date_folder = get_date_folder(full_path)
                destination = os.path.join(path, date_folder)
                create_folder(destination)
                move_file(full_path, destination)

In [ ]:
def organize_by_keyword(path, keywords):
    # IMPORTANT: Ensure that 'keywords' list is not empty and contains relevant keywords
    # for the code to correctly categorize files based on their names.
    for root, _, files in os.walk(path):
        if any(root.endswith(palavra) for palavra in keywords) or root.endswith("Others"): continue
        for file in files:
            name = file.lower()
            full_path = os.path.join(root, file)
            if not os.path.isfile(full_path): continue
            found = False
            for keyword in keywords:
                if keyword.lower() in name:
                    destination = os.path.join(path, keyword)
                    create_folder(destination)
                    move_file(full_path, destination)
                    found = True
                    break
            if not found:
                destination = os.path.join(path, "Others")
                create_folder(destination)
                move_file(full_path, destination)

## ▶️ Step 4 - Execute

Choose one or more organization functions to run and organize the files inside a specific folder in your Google Drive.

You can use, for example:

- `organize_by_type("/content/drive/MyDrive/Projects")`

- `organize_by_size("/content/drive/MyDrive/Downloads")`

- `organize_by_date("/content/drive/MyDrive/Tasks")`

- `organize_by_keyword("/content/drive/MyDrive/Backup", ["exam", "list", "assignment"])`

> ✅ Tip: replace the path `/content/drive/MyDrive` with the specific folder you want to organize. This keeps the process safer and more controlled, without reorganizing everything.

---

## ▶️ Etapa 4 - Executar

Escolha uma ou mais funções de organização para executar e organizar os arquivos dentro de uma pasta específica do seu Google Drive.

Você pode usar, por exemplo:

- `organizar_por_tipo("/content/drive/MyDrive/Trabalhos")`

- `organizar_por_tamanho("/content/drive/MyDrive/Downloads")`

- `organizar_por_data("/content/drive/MyDrive/Projetos")`

- `organizar_por_palavra_chave("/content/drive/MyDrive/Backup", ["prova", "lista", "trabalho"])`

> ✅ Dica: substitua o caminho `/content/drive/MyDrive` pela pasta que você quer organizar. Isso torna o processo mais seguro e controlado, sem reorganizar arquivos desnecessários.


In [ ]:
my_drive = "/content/drive/MyDrive"

# WARNING: uncomment to execute!

# organize_by_type(my_drive)
# organize_by_size(my_drive)
# organize_by_date(my_drive)
# organize_by_keyword(my_drive, ["prova", "lista", "trabalho"])  # customizable list/lista personalizável